In [1]:
# 20171615 최하록

## HW6

In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [3]:
MIN_MATCH_COUNT = 10
FLANN_INDEX_KDTREE = 0
K = 2

index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

template = cv2.imread('Temple.jpg', 0)
cap = cv2.VideoCapture(0)

cv2.namedWindow('image')
cv2.createTrackbar('Threshold', 'image', 0, 100, lambda x: None)
cv2.setTrackbarPos('Threshold', 'image', 60)

sift = cv2.SIFT_create()
flann = cv2.FlannBasedMatcher(index_params, search_params)

while cap.isOpened():
    ret, frame = cap.read()
    
    threshold = cv2.getTrackbarPos('Threshold','image') / 100.0

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    kp1, des1 = sift.detectAndCompute(gray, None)
    kp2, des2 = sift.detectAndCompute(template, None)
    
    matches = flann.knnMatch(des1, des2, k=K)
    good = [m for (m, n) in matches if (m.distance < threshold * n.distance)]
    
    matchesMask = None
            
    if len(good) > MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1, 1, 2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1, 1, 2)
        
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        matchesMask = mask.ravel().tolist()
        
        h,w = gray.shape
        
        pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0] ]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, M)
        gray = cv2.polylines(gray, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)
        
    draw_params = dict(matchColor = (0, 0, 255), singlePointColor = None, matchesMask = matchesMask, flags = 2)
    result = cv2.drawMatches(gray, kp1, template, kp2, good, None, **draw_params)

    cv2.imshow("image", result)

    if cv2.waitKey(40) & 0xFF == ord('s'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [4]:
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [1]:
# 고생많으셨습니다! 교수님!